In [1]:
import numpy as np
import scipy.stats as stats

In [69]:
class EuropeanRainbow:
    
    def __init__(self):
        
        pass
    
    def get_Stulz_price(self, S = None, cov = None, r = None, K = None, matu = None, N = None, nbPaths = None):
        
        dt = matu / N
        T = np.arange(0,N) * dt
        T = np.repeat(np.flip(T[None,:]), S.shape[0], axis=0)
        
        spot1 = np.zeros((nbPaths,N))
        spot2 = np.zeros((nbPaths,N))
        
        eta1 = np.zeros((nbPaths,N))
        eta2 = np.zeros((nbPaths,N))
        
        beta1 = np.zeros((nbPaths,N))
        beta2 = np.zeros((nbPaths,N))
        
        gamma1 = np.zeros((nbPaths,N))
        gamma2 = np.zeros((nbPaths,N))
        
        spot1[:,:] = S[:,:,0]
        spot2[:,:] = S[:,:,1]
        
        sigma1 = np.sqrt(cov[0,0])
        sigma2 = np.sqrt(cov[1,1])
        rho = cov[0,1]/(sigma1*sigma2)
        
        sig = sigma1**2 + sigma2**2 + 2*rho*sigma1*sigma2
        rho_indiv1 = (rho*sigma2 - sigma1) / sig
        rho_indiv2 = (rho*sigma1 - sigma2) / sig
        
        with np.errstate(divide="ignore"):
            eta1 = np.divide(np.log(spot1/K)+(r+0.5*sigma1**2)*T, (sigma1*np.sqrt(T)))
            eta2 = np.divide(np.log(spot2/K)+(r-0.5*sigma2**2)*T, (sigma2*np.sqrt(T)))
            
            beta1 = np.divide(np.log(spot1/spot2)-0.5*(sig**2*np.sqrt(T)),sig*np.sqrt(T))
            beta2 = np.divide(np.log(spot2/spot1)-0.5*(sig**2*np.sqrt(T)),sig*np.sqrt(T))
            
            gamma1 = eta1 - sigma1*np.sqrt(T)
            gamma2 = eta2 - sigma2*np.sqrt(T)
            
        price = spot1 * np.random.multivariate_normal((eta1[0,0],beta1[0,0]),rho_indiv1)
        
        return price

m=np.array([0.1,0.2])
spot_init=np.array([100,100])
T=1
N=30
cov=np.array([[0.1,0.05],[0.05,0.1]])
prob1=0.2
prob2=0.2
r=0.0
K=100
matu=1
nbPaths=1

test = multiJumpDiffusion(m=m, spot_init=spot_init, T=T, N=N, cov=cov, prob1=prob1, prob2=prob2)
S = test.gen_path(nbPaths=nbPaths)

rainbow = EuropeanRainbow()
spot_divide = rainbow.get_Stulz_price(S=S,cov=cov,r=r,K=K,matu=matu,N=N,nbPaths=nbPaths)
# so far I managed to code something that separates well the dynamics of the two underlying assets!

print(spot_divide)

<class 'ValueError'>: cov must be 2 dimensional and square

In [70]:
# multi-asset jump diffusion model

import numpy as np
import matplotlib.pyplot as plt

class multiJumpDiffusion:
    
    def __init__(self, m = None, spot_init = None, T = None, N = None, cov = None, prob1 = None, prob2 = None):
        
        self.m = m
        self.spot_init = spot_init
        self.T = T
        self.N = N
        self.cov = cov
        self.prob1 = prob1
        self.prob2 = prob2
        
    def gen_process(self):
        
        dt = self.T / self.N
        sigma = np.zeros(2)
        processes = np.zeros((self.N, 2))
        processes[0,:] = np.log(self.spot_init)
        
        drift = np.zeros(2)
        diffusion = np.zeros(2)
        
        for i in range(1, self.N):
            
            z1 = np.random.normal(0,1)
            sigma[0] = np.sqrt(cov[0,0])
            sigma[1] = np.sqrt(cov[1,1])
            
            drift = 0.5 * (sigma ** 2) * dt
            diffusion = np.sqrt(self.T / self.N) * np.matmul(np.linalg.cholesky(self.cov), np.random.normal(0,1,size=2))
            jump = self.gen_jump(sigma[0], sigma[1])
            
            processes[i,:] = processes[i-1,:] - drift + diffusion + jump
            
        return np.exp(processes)
    
    def gen_jump(self, sigma1 = None, sigma2 = None):
        
        jump = np.zeros(2)
        a1 = np.zeros(2)
        a2 = np.zeros(2)
        a3 = np.zeros(2)
        zero = np.zeros(2)
        
        a1[:] = 0
        a2[:] = 0
        a3[:] = 0
        
        z=np.zeros(3)
        z[0] = 1
        z[1] = 2
        z[2] = 3
        
        val = np.random.choice(z,1,p=[1 - self.prob1 - self.prob2, self.prob1, self.prob2])
        
        if val == 1:
            jump = (self.m*np.random.exponential()+np.sqrt(np.random.exponential())*np.random.multivariate_normal(zero,self.cov))
        elif val == 2:
            jump = [sigma1*np.random.exponential()+np.sqrt(np.random.exponential())*np.random.normal(0,sigma1**2),0]
        elif val == 3:
            jump = [0,sigma2*np.random.exponential()+np.sqrt(np.random.exponential())*np.random.normal(0,sigma2**2)]
        
        return np.random.poisson(1*self.T/self.N,size=2)*jump
    
    def gen_path(self, nbPaths = None):
        
        dt = np.divide(self.T,self.N)
        dualPaths = np.zeros(((nbPaths,self.N,2)))
        
        for i in range(nbPaths):
            dualPaths[i,:,:] = self.gen_process()
        
        return dualPaths
    
m=np.array([0.1,0.2])
spot_init=np.array([100,100])
T=1
N=30
cov=np.array([[0.1,0.05],[0.05,0.1]])
prob1=0.2
prob2=0.2
    
test = multiJumpDiffusion(m=m, spot_init=spot_init, T=T, N=N, cov=cov, prob1=prob1, prob2=prob2)
hihi = test.gen_path(3)

In [73]:
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

from math import log, sqrt, exp, pi

In [74]:
# looks like a cdf implementation: https://github.com/stan-dev/stan/issues/2356
# looks like it is talked about in more details here: https://github.com/tensorflow/probability/issues/422

# bon apparement on peut le faire en matlab mdr: https://fr.mathworks.com/matlabcentral/fileexchange/7025-bivariate-cumulative-normal-probability
# c'est déjà ça

# voila une implémentation python à étudier: https://github.com/wanglouis49/risk_estimation/blob/master/blspricer.py
# autre github qui traite de cette fonction: https://github.com/nuance/python-nlp/blob/master/nlp/lib/mvncdf.py
# mmmmh intéressant! - http://www-2.rotman.utoronto.ca/~hull/TechnicalNotes/TechnicalNote5.pdf

In [93]:
def bivnormcdf(a,b,rho):

	if a <= 0. and b <= 0. and rho <= 0. :
		aprime = a/sqrt(2.*(1.-rho**2.))
		bprime = b/sqrt(2.*(1.-rho**2.))
		A = np.array([0.3253030, 0.4211071, 0.1334425, 0.006374323])
		B = np.array([0.1337764, 0.6243247, 1.3425378, 2.2626645])

		t = 0.

		for i in range(4):
			for j in range(4):
				x = B[i]
				y = B[j]
				t += A[i]*A[j]* exp(aprime*(2.*x - aprime) \
				     + (bprime*(2.*y - bprime)) + (2.*rho * (x - aprime)*(y-bprime)))

		p = (sqrt(1.-rho**2.)/pi) * t

	elif a * b * rho <= 0. :
		if a <= 0. and b >= 0. and rho >= 0. :
			p = stats.norm.cdf(a) - bivnormcdf(a,-b,-rho)
		elif a >= 0. and b <= 0. and rho >= 0. :
			p = stats.norm.cdf(b) - bivnormcdf(-a,b,-rho)
		elif a >= 0. and b >= 0. and rho <= 0. :
			p = stats.norm.cdf(a) + stats.norm.cdf(b) - 1. + bivnormcdf(-a,-b,rho)

	elif a*b*rho > 0. :
		if a >= 0. :
			asign = 1.
		else:
			asign = -1.

		if b >= 0.:
			bsign = 1.
		else:
			bsign = -1.

		rho1 = (rho*a - b)*asign/(sqrt(a**2. - (2.*rho*a*b) + b**2.))
		rho2 = (rho*b - a)*bsign/(sqrt(a**2. - (2.*rho*a*b) + b**2.))
		delta = (1. - (asign*bsign))/4.

		p = bivnormcdf(a,0,rho1) + bivnormcdf(b,0,rho2) - delta
	return p

a = bivnormcdf(0,0,0.99)
print(a)

# on se base sur le papier de Hull
# je pense que ce soir on peut essayer d'implémenter tout ça sur notre gros code Python
# puis implémenter tout ça (concisement bien sur) dans notre papier
# essayer de refaire tourner notre algo de vanille et tenter de trouver le pb (pk il marche pas)
# continuer à candidater à plein de stages (et bien tenir le excel avec l'historique des candidatures)

# faut absolument qu'on ait un code final parfait et très très bien structuré!!

0.4774746939518141


In [96]:
# résumé de l'avancé de la journée

# très légères recherches sur les signaux de trading liés à la volat et à la correl (besoin de mieux structurer tout ça)
# on a bien avancer sur le multi-JD (je pense qu'il est carré), et je pense donc que toutes nos dynamiques sont plus ou moins bonnes
# on a plutot avancé sur la classe des Rainbows, on y voit plus clair
# on a réussi à séparer le set initial de deux spots en deux sets d'un spot (utile pour la classe Rainbow)
# on a ENFIN trouvé comment calculer dans Python la proba bivariate std normale, qui nous sera ultra utile pour tout!

# plan de la poursuite du travail de ce soir (hors candidatures pour stages)

# implémentation très très propre de tous nos ajouts dans le gros dossier de coode sur PyCharm
# on tente de continuer la classe des Rainbow (au moins le prix et le delta de l'option) + on s'assure que c'est pas aberrant
# on vérifie que l'implémentation de toutes les dynamiques nous fournit des résultats cohérents, pour pouvoir avoir bouclé cte partie
# on implémente nos changements par écrit dans le papier (de manière propre) et on fait qq modifs de clarté dans le chap5
# refaire tourner un peu notre algo et tenter de comprendre pk les résultats sont bizarres pour la stratégie de deep hedge

# plan de candidatures aux stages de ce soir

# minimum de cinq belles candidatures (on tente huit)
# on y va à fond!